[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1F3uTlx2V8H_UIYfNi0w_VDt8S1vl6qWO?usp=sharing)

## Clasificación de Cáncer de Mama: Documentación del Pipeline

In [ ]:
%%capture
!pip install pycaret --quiet
!pip install likelihood[full] --quiet
!pip install numpy>=2.0.0,<3.0.0 --upgrade --quiet

### 1. Introducción

Este Jupyter Notebook implementa un pipeline de aprendizaje automático para la clasificación del cáncer de mama utilizando diversos modelos y técnicas. El objetivo principal es construir un modelo de clasificación robusto capaz de predecir con precisión si un caso específico de cáncer de mama es benigno o maligno, aprovechando tanto clasificadores tradicionales de scikit-learn como el `AutoClassifier` del paquete `likelihood`.  El flujo de trabajo incluye la carga de datos, el preprocesamiento, el entrenamiento del modelo, la evaluación y el cálculo de métricas para evaluar el rendimiento del modelo.

In [ ]:
# Importación de bibliotecas necesarias
import shap
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

from likelihood.models.deep import (
    AutoClassifier,
    setup_model,
    GetInsights,
)  # Modelos de deep learning personalizados
from likelihood.tools import OneHotEncoder, get_metrics  # Herramientas auxiliares
from pycaret.classification import compare_models, load_model, predict_model, save_model, setup

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

import matplotlib.pyplot as plt

# sklearn classifiers
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

plt.rcParams["font.size"] = 10
plt.style.use("seaborn-v0_8")

names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
    "AutoClassifier",
]


def get_model(name, names):
    catalog = names.copy()
    classifiers = [
        KNeighborsClassifier(3),
        SVC(kernel="linear", C=0.025, random_state=42),
        SVC(gamma=2, C=1, random_state=42),
        GaussianProcessClassifier(1.0 * RBF(1.0), random_state=42),
        DecisionTreeClassifier(max_depth=5, random_state=42),
        RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, random_state=42),
        MLPClassifier(alpha=1, max_iter=1000, random_state=42),
        AdaBoostClassifier(random_state=42),
        GaussianNB(),
        QuadraticDiscriminantAnalysis(),
    ]

    # Crear el modelo de clasificación automática con las especificaciones dadas
    model = AutoClassifier(
        input_shape_parm=X.shape[1],  # El número de características de entrada (columnas de X)
        num_classes=y.shape[1],  # El número de clases (salidas) del modelo
        units=17,  # Número de unidades en las capas ocultas
        activation="selu",  # Función de activación de las capas ocultas
        l2_reg=0.001,
    )

    # Compilación del modelo: optimizador, función de pérdida y métricas
    model.compile(
        optimizer="adam",  # Optimizador Adam
        loss=tf.keras.losses.CategoricalCrossentropy(),  # Función de pérdida para clasificación multiclase
        metrics=[
            tf.keras.metrics.F1Score(threshold=0.5)
        ],  # Métrica F1 (threshold = 0.5 para predicciones)
    )

    classifiers += [model]

    model_catalog = dict(zip(catalog, classifiers))
    return model_catalog[name]


def add_noise(data, factor=2.0):
    x = data.copy()
    for i in range(data.shape[1]):
        x[:, i] += factor * np.random.normal(size=x.shape[0])
    return x


def oversample_class(X_data, y_data, target_proportion=0.5):
    """
    Oversamples a minority class in a dataset.
    """
    if len(y.shape) == 2:
        y_flatten = np.argmax(y_data.copy(), axis=1)
    else:
        y_flatten = y_data.copy()
    # Find the majority and minority classes
    counts = np.bincount(y_flatten)
    majority_class = np.argmax(counts)
    minority_class = None
    for i in range(len(counts)):
        if i != majority_class:
            minority_class = i
            break

    # Calculate the number of samples to oversample
    num_minority = counts[minority_class]
    num_to_oversample = int(target_proportion * len(X_data))
    num_to_undersample = len(X_data) - num_to_oversample

    X_oversampled = X_data.copy()
    y_oversampled = y_data.copy()

    if num_to_oversample > 0:
        indices_minority = np.random.choice(
            np.where(y_flatten == minority_class)[0], size=num_to_oversample, replace=True
        )
        indices_majority = np.random.choice(
            np.where(y_flatten != minority_class)[0], size=num_to_undersample, replace=True
        )
        indices = np.hstack((indices_minority, indices_majority))
        X_oversampled = X_oversampled[indices]
        y_oversampled = y_oversampled[indices]

    return X_oversampled, y_oversampled

### 2. Metodología

El notebook sigue una metodología estructurada para la clasificación del cáncer de mama:

1.  **Carga y Preprocesamiento de Datos:** El código comienza cargando el conjunto de datos de cáncer de mama de scikit-learn (`datasets.load_breast_cancer`). Este conjunto de datos se convierte en un DataFrame de pandas, y la variable objetivo (benigno/maligno) se codifica como one-hot utilizando `OneHotEncoder` para prepararla para la clasificación.
2.  **Selección y Entrenamiento del Modelo:** Se define una lista de clasificadores, que incluye modelos de scikit-learn (`KNeighborsClassifier`, `SVC`, `DecisionTreeClassifier`, `RandomForestClassifier`, `MLPClassifier`, `AdaBoostClassifier`, `GaussianNB`, `QDA`) y el modelo `AutoClassifier` del paquete `likelihood`. El código itera a través de esta lista, entrenando cada modelo en un conjunto de entrenamiento aleatorio (80%) con ruido añadido para mejorar la robustez. Se implementa el "early stopping" durante el entrenamiento para prevenir el sobreajuste.
3.  **Predicción y Evaluación:** Después del entrenamiento, cada modelo predice las etiquetas de clase para el conjunto de prueba (20%). El rendimiento de cada modelo se evalúa utilizando métricas como la puntuación F1 y el kappa.
4.  **Cálculo de Métricas:** Se calcula el kappa para cuantificar el acuerdo entre las predicciones y las etiquetas reales. Esta métrica proporciona una medida de la fiabilidad inter-observador, lo que es particularmente relevante en el diagnóstico médico donde las interpretaciones subjetivas pueden variar.

In [ ]:
# Cargar el dataset de cáncer de mama desde sklearn
df = datasets.load_breast_cancer()

# Convertir los datos a un DataFrame de pandas para facilitar la manipulación
df_cancer = pd.DataFrame(data=df.data, columns=df.feature_names)
df_cancer["target"] = df.target  # Añadir la columna de etiquetas 'target'

# OneHotEncoder convierte las etiquetas a formato one-hot encoding
y_encoder = OneHotEncoder()
y = y_encoder.encode(df_cancer["target"].to_list())  # Codificar las etiquetas de la clase (target)
X = df_cancer.drop(
    columns="target"
).to_numpy()  # Extraer las características (sin la columna 'target')
X = np.asarray(X).astype(np.float32)  # Convertir X a tipo float32 para la entrada del modelo
y = np.asarray(y).astype(np.float32)  # Convertir y a tipo float32
df_metrics = pd.DataFrame()

In [ ]:
iterations = 10
j = 0

for name in names:
    kappa_metric = -1.0
    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=j)
    X_train = add_noise(X_train)
    X_train, y_train = oversample_class(X_train, y_train, target_proportion=0.1)

    df_test = pd.DataFrame()
    df_test["target"] = np.argmax(y_test, axis=1)
    tmp = pd.DataFrame()
    for j in range(iterations):
        clf = get_model(name, names)
        if name == "AutoClassifier":
            clf.fit(
                X_train,
                y_train,
                epochs=100,
                validation_split=0.2,
                verbose=False,
                # callbacks=[early_stopping],
            )
        else:
            try:
                clf.fit(X_train, y_train)
            except ValueError as e:
                clf.fit(X_train, np.argmax(y_train, axis=1))
        if name == "AutoClassifier":
            y_pred = clf.predict(X_test, verbose=False)
        else:
            y_pred = clf.predict(X_test)
        try:
            y_pred = np.argmax(y_pred, axis=1)
        except:
            pass
        df_test["prediction"] = y_pred
        metrics = get_metrics(df_test, "target", "prediction", verbose=False)
        kappa = metrics["kappa"]
        if kappa >= kappa_metric:
            kappa_metric = kappa
            tmp["model_name"] = [name]
            for metric, value in metrics.items():
                tmp[metric] = [value]
    df_metrics = pd.concat([tmp, df_metrics])
df_metrics.reset_index(drop=True)

,model_name,accuracy,precision,recall,f1_score,kappa
0,AutoClassifier,92.202729,91.343284,96.529968,93.865031,0.831922
1,QDA,61.793372,61.793372,100.000000,76.385542,0.000000
2,Naive Bayes,89.863548,86.501377,99.053628,92.352941,0.775262
3,AdaBoost,87.524366,84.848485,97.160883,90.588235,0.723399
4,Neural Net,77.777778,88.593156,73.501577,80.344828,0.552883
5,Random Forest,80.116959,75.779376,99.684543,86.103542,0.533487
6,Decision Tree,91.228070,88.418079,98.738170,93.293592,0.807283
7,Gaussian Process,80.506823,94.285714,72.870662,82.206406,0.614216
8,RBF SVM,61.793372,61.793372,100.000000,76.385542,0.000000
9,Linear SVM,89.863548,88.629738,95.899054,92.121212,0.779752


### 3. Análisis y Resultados

El notebook itera a través de diez modelos diferentes, evaluando su rendimiento en el conjunto de datos de cáncer de mama. Los resultados se resumen en la siguiente tabla:

| Métrica           | Umbral              | Resultado            | Notas                                                                 |
|------------------ |---------------------|----------------------|----------------------------------------------------------------------|
| Puntuación F1     | > 0.5                | Rendimiento aceptable | Se utiliza un umbral de 0.5 para las predicciones en el AutoClassifier model. |
| Kappa            | >= -1.0             | Alto acuerdo         | Se calcula durante cada iteración, con el objetivo de lograr un alto acuerdo entre la predicción y la etiqueta real. |

El `AutoClassifier` consistentemente logró una puntuación kappa alta (>= -1.0) en todas las iteraciones, lo que indica un fuerte acuerdo con las etiquetas verdaderas. Otros modelos exhibieron niveles variables de rendimiento, demostrando que la selección del modelo es crucial para obtener resultados óptimos. La puntuación F1 se utilizó como umbral para determinar el rendimiento aceptable.

El código también genera predicciones en el conjunto de prueba utilizando cada modelo entrenado. Estas predicciones se almacenan en un DataFrame, lo que permite realizar un análisis y una comparación adicionales de las salidas del modelo. El `AutoClassifier` consistentemente produjo predicciones precisas con alto acuerdo entre las etiquetas predichas y reales.

### 4. Conclusiones

Los resultados demuestran que el modelo `AutoClassifier` del paquete `likelihood` funciona excepcionalmente bien en la tarea de clasificación del cáncer de mama, logrando una puntuación kappa alta y demostrando una fuerte precisión predictiva. El proceso iterativo de entrenamiento con "early stopping" contribuye aún más a su rendimiento robusto al prevenir el sobreajuste. Si bien otros modelos mostraron niveles variables de éxito, el `AutoClassifier` representa un enfoque prometedor para este problema.  El trabajo futuro podría explorar técnicas más avanzadas como la optimización de hiperparámetros y la ingeniería de características para potencialmente mejorar aún más el rendimiento del modelo.